## **Hybrid Architecture: Fusion Retrieval + Reranking based RAG** 

_**What are your expectations for the Hybrid architecture with the provided manual?**_<br>
The obvious answer would be that it will be better than both the approaches. Hybrid models in general tend to perform better than their individual constituent models. But this requires testing.

**_How do you plan to test and compare these techniques?_**<br><br>
<img src="./hybrid_workflow.png" alt="Flowchart" width="700" /><br><br>
The addition here as compared to the Fusion Retrieval architecture is the addition of an LLM based reranking approach right before asking an LLM to generate the response. In this case, there will be 3 sets of chunks throughout the workflow. First will be the set that is returned from the initial BERT embeddings and BM25 representations. This will be denoted as top-K. The next set of chunks will be based on the fusion scores from the semantic based retried chunks and keyword based retrieved chunks. This will be denoted as top-L. And finally, this set of L chunks will be passed into the first LLM for reranking. The chunks returned here will be the top-M chunks which will be the final set of context for our second and final LLM for query answering.<br>
1. The document data is extracted, specifically text and tabular data. 
2. Next, these data are stored in a way where the sequence is maintained, that way there will be more context for a certain text that may have a table before or after it. 
3. This set is chunked and converted to BERT based vector embeddings and also to BM25 based representations.
4. Now the query is also converted to BERT based embeddings and BM25 based representations and the top-K chunks are retrieved for both. A union is taken between the retrieved chunks to avoid overlaps.
5. Next, this final set of chunks are again made into 2 copies, one of BERT based embeddings and the other of BM25 based inverted indexes. The query is passed into both structures and the scores are retrieved for each chunk. This gives us a set of chunk IDs, BERT scores, BM25 scores and the final fusion score. 
6. Based on this fusion score, the top-L chunks are retrieved (L < K). 
7. These top-L chunks are passed into the first LLM for reranking, which returns the top-M chunks based on its knowledge and understanding (M < L).
8. This final set of top-M chunks are passed into the second LLM, along with the query for the final response generation.

*Note: K > L > M*

_**Comparison Strategy**_<br>
There are possibly two main ways in which we can compare this approach with the Fusion Retrieval and Reranking approach. One is by assessing the top-M retrieved chunks from the Hybrid with the top-L retrieved chunks from both the other approaches. The other is obviously by assessing the final response from the LLM for each approach. 

---

#### Import libraries

In [2]:
import fitz # for text extraction
import camelot # for table extraction
from sentence_transformers import SentenceTransformer, util # for semantic vector embedding creation 
from rank_bm25 import BM25Okapi # for bm25 implementation
import spacy # for stop word removal
import re
from pathlib import Path
import numpy as np
from groq import Groq
import os
import time
import ollama

#### 1. Function to extract texts & tables from PDF
The goal is to preserve the sequence, that way there will be more context for a certain text that may have a table before or after it.

In [3]:
def extract_text_and_tables(pdf_path):

    pdf_file = Path(pdf_path)
    if not pdf_file.is_file() or pdf_file.suffix.lower() != ".pdf":
        raise FileNotFoundError("Provided file path is not a valid PDF.")

    doc = fitz.open(str(pdf_file))
    result = []

    # text extraction
    for page_num, page in enumerate(doc, start = 1):
        page_blocks = []

        blocks = page.get_text("dict")["blocks"]
        for block in blocks:
            if block["type"] == 0: # type 0 is text
                text_content = " ".join(
                    span["text"] for line in block["lines"] for span in line["spans"]
                ).strip()
                if text_content:
                    y = block["bbox"][1]
                    page_blocks.append({
                        "type": "TEXT DATA",
                        "y": y,
                        "content": text_content
                    })

        # table extraction
        try:
            tables = camelot.read_pdf(str(pdf_file), pages = str(page_num), flavor = 'lattice') # lattice flavor to extract tables
        except Exception as e:
            print(f"Failed to read tables on page {page_num}: {e}")
            tables = []

        for table in tables:
            table_data = table.data
            bbox = table._bbox
            y = float(bbox[1])
            page_blocks.append({
                "type": "TABLE DATA",
                "y": y,
                "content": table_data
            })

        page_blocks.sort(key=lambda b: b["y"]) # sort contents on current page
        result.extend(page_blocks) # append content to result list

    return result

In [4]:
# extract texts and tables from the maual
result = extract_text_and_tables("manual.pdf")
result[100:105] # few elements from the extracted data

[{'type': 'TEXT DATA',
  'y': 145.75482177734375,
  'content': 'Follow the instructions contained herein, in addition to the general precautions to be observed while working. Even if the operator is already familiar with the use of manually operated lathes, it is necessary to: In particular:'},
 {'type': 'TEXT DATA', 'y': 173.48190307617188, 'content': 'fervi.com'},
 {'type': 'TEXT DATA',
  'y': 188.8348388671875,
  'content': '\uf0b7 Acquire full knowledge of the machine. For safe operation, this manual must be read carefully in order to acquire the necessary knowledge of the machine and to understand: operation, safety devices and all necessary precautions. \uf0b7 Wear appropriate clothing for the job. The operator must wear appropriate clothing to prevent accidents. \uf0b7 Maintain the machine with care.'},
 {'type': 'TEXT DATA',
  'y': 312.05987548828125,
  'content': 'Risks associated with using the machine'},
 {'type': 'TEXT DATA',
  'y': 342.43487548828125,
  'content': 'The mac

In [5]:
# sample table data
result[156]

{'type': 'TABLE DATA',
 'y': 91.1826731262468,
 'content': [['Description (unit of measurement)', 'T999/230V\nT999/400V'],
  ['Centres distance (mm)', '1000'],
  ['Spindle hole diameter (mm)', '38'],
  ['Maximum swing over the bed (mm)', '320'],
  ['Maximum swing over the cross slide (mm)', '198'],
  ['Turning diameter over cavity (mm)', ''],
  ['Spindle diameter (3 + 3 self centring) (mm)', ''],
  ['Spindle connector', ''],
  ['No. of spindle speeds', 'm'],
  ['Spindle speed (r/min)', ''],
  ['No. of metric threads', ''],
  ['Range of metric threads (mm)', 'o'],
  ['No. of inch threads', ''],
  ['Range of inch threads (mm)', ''],
  ['Range of longitudinal\nfeeds (mm)', '00.78- 1.044\nc'],
  ['Range of transverse feeds (mm)', '0.022- 0.298'],
  ['Outer diameter of the feed screw (mm)\n.', '22'],
  ['Guide length (mm)\ni', '1390'],
  ['Cross carriage travel (mm)\nv', '200'],
  ['Tailstock sleeve diameter (mm)', '32'],
  ['Maximum travel of the tailstock sleeve (mm)\nr', '80'],
  ['Inner

In [6]:
# list formatting by adding labels for texts and tables
final = []
for r in result:
    s = f"{r['type']}: {r['content']}"
    final.append(s)

In [7]:
# table data sample after flattening
final[156]

'TABLE DATA: [[\'Description (unit of measurement)\', \'T999/230V\\nT999/400V\'], [\'Centres distance (mm)\', \'1000\'], [\'Spindle hole diameter (mm)\', \'38\'], [\'Maximum swing over the bed (mm)\', \'320\'], [\'Maximum swing over the cross slide (mm)\', \'198\'], [\'Turning diameter over cavity (mm)\', \'\'], [\'Spindle diameter (3 + 3 self centring) (mm)\', \'\'], [\'Spindle connector\', \'\'], [\'No. of spindle speeds\', \'m\'], [\'Spindle speed (r/min)\', \'\'], [\'No. of metric threads\', \'\'], [\'Range of metric threads (mm)\', \'o\'], [\'No. of inch threads\', \'\'], [\'Range of inch threads (mm)\', \'\'], [\'Range of longitudinal\\nfeeds (mm)\', \'00.78- 1.044\\nc\'], [\'Range of transverse feeds (mm)\', \'0.022- 0.298\'], [\'Outer diameter of the feed screw (mm)\\n.\', \'22\'], [\'Guide length (mm)\\ni\', \'1390\'], [\'Cross carriage travel (mm)\\nv\', \'200\'], [\'Tailstock sleeve diameter (mm)\', \'32\'], [\'Maximum travel of the tailstock sleeve (mm)\\nr\', \'80\'], [\'I

It can be seen that the flattened version somewhat preserves the structure of the actual table by keeping each row inside a list. The LLM can hopefully understand this due to the presence of the label 'TABLE DATA' at the start.

### 2. Chunking

In [8]:
chunks = [" ".join(final[i:i+10]) for i in range(0, len(final), 30)]
print(f"Number of chunks: {len(chunks)}\n")

chunks[15] # sample

Number of chunks: 46



'TEXT DATA: 8.3 Levelling the machine TEXT DATA: For this operation, it is recommended to use a precision spirit level (0.001 mm). TEXT DATA: 8.3.1 Preliminary phase TEXT DATA: The preliminary phase serves to eliminate the presence of torsions in the lathe table. Proceed to reset the head by adjusting the relative screws and then locking the tailstock with the relative adjustment screws moving the reference mark to zero. TEXT DATA: fervi.com TEXT DATA: 8.3.2 Transverse levelling of the table TEXT DATA: Position the spirit level in a transverse direction on the lathe guides under the spindle and check the bubble. Position the spirit level in a transverse direction on the table guides under the tailstock and check the bubble. Repeat these operations frequently and, if necessary, make small corrections by screwing and/or unscrewing the adjustable feet below the pallet. TEXT DATA: 8.3.3 Levelling the lathe rails TEXT DATA: Place the level on the sides of the carriage and move it slowly alo

### 3. Data cleaning for BM25

In [9]:
nlp = spacy.load("en_core_web_sm")

chunks_4_bm25 = []
for chunk in chunks:
    doc = nlp(chunk)
    filtered = [token.text for token in doc if not token.is_stop]
    chunks_4_bm25.append(" ".join(filtered))

chunks_4_bm25[15] # sample

'TEXT DATA : 8.3 Levelling machine TEXT DATA : operation , recommended use precision spirit level ( 0.001 mm ) . TEXT DATA : 8.3.1 Preliminary phase TEXT DATA : preliminary phase serves eliminate presence torsions lathe table . Proceed reset head adjusting relative screws locking tailstock relative adjustment screws moving reference mark zero . TEXT DATA : fervi.com TEXT DATA : 8.3.2 Transverse levelling table TEXT DATA : Position spirit level transverse direction lathe guides spindle check bubble . Position spirit level transverse direction table guides tailstock check bubble . Repeat operations frequently , necessary , small corrections screwing and/or unscrewing adjustable feet pallet . TEXT DATA : 8.3.3 Levelling lathe rails TEXT DATA : Place level sides carriage slowly entire length checking bubble change . bubble moves , adjust adjustable feet reaches uniform level entire course carriage . Periodically check measurements ( months ) . TEXT DATA : Levelling machine perfectly essent

### 4. Creating semantic vector embeddings and BM25 inverted index

In [10]:
# for bert
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
sem_embs = model.encode(chunks, convert_to_tensor = True)

In [11]:
# for bm25
tokenized_corpus = [doc.split() for doc in chunks_4_bm25]
bm25 = BM25Okapi(tokenized_corpus)

### 5. Pipeline to return indices of top-K chunks that match with the query

In [ ]:
def bert_query_pipeline(query, top_k = 30):
     
    device = sem_embs.device
    query_embedding = model.encode(query, convert_to_tensor = True)
    cosine_scores = util.cos_sim(query_embedding, sem_embs)[0] # cosine similarity
    top_indices = np.argsort(cosine_scores.cpu().numpy())[::-1][:top_k]

    return top_indices

In [ ]:
def bm25_query_pipeline(query, top_k = 30):

    tokenized_query = query.split()
    bm25_scores = bm25.get_scores(tokenized_query) # tf-idf like scoring
    top_indices = np.argsort(bm25_scores)[::-1][:top_k]

    return top_indices

In [14]:
# query = 'What are some general safety rules when using machine equipment?'
query = 'What does the manual say about unplugging the power cord of the machine from the power outlet?'

In [15]:
# get common chunks from chunks retrived from both implementations

bert_top_k_idx = bert_query_pipeline(query)
bm25_top_k_idx = bm25_query_pipeline(query)
final_idx = list(set(list(bert_top_k_idx) + list(bm25_top_k_idx))) # union operation

staged_context = [chunks[idx] for idx in final_idx]
staged_context_4_bm25 = [chunks_4_bm25[idx] for idx in final_idx]
print(f"Number of staged chunks for context: {len(staged_context)}\n")

Number of staged chunks for context: 23



### 6. Embed staged context using BERT & get inverted indices of staged context using BM25

In [16]:
# for bert
sem_embs_final = model.encode(staged_context, convert_to_tensor = True)

# for bm25
tokenized_corpus_final = [doc.split() for doc in staged_context_4_bm25]
bm25_final = BM25Okapi(tokenized_corpus_final)

### 7. Function to get the final set of scores for the staged context chunks for both BERT & BM25

In [17]:
def bert_final_scores(query):
     
    device = sem_embs_final.device
    query_embedding = model.encode(query, convert_to_tensor = True)
    cosine_scores = util.cos_sim(query_embedding, sem_embs_final)[0]
    indices = np.argsort(cosine_scores.cpu().numpy())[::-1]

    return cosine_scores.cpu().numpy(), indices

In [18]:
bert_final_scores(query)

(array([0.36677617, 0.27207935, 0.38090566, 0.45139548, 0.27475065,
        0.27165347, 0.30504328, 0.45442048, 0.11290099, 0.4188665 ,
        0.3742811 , 0.23033723, 0.43445706, 0.26141763, 0.28430384,
        0.24624479, 0.34687048, 0.33071345, 0.1132399 , 0.32948324,
        0.31769413, 0.2676775 , 0.29725122], dtype=float32),
 array([ 7,  3, 12,  9,  2, 10,  0, 16, 17, 19, 20,  6, 22, 14,  4,  1,  5,
        21, 13, 15, 11, 18,  8], dtype=int64))

In [19]:
def bm25_query_pipeline(query):

    tokenized_query = query.split()
    bm25_scores = bm25_final.get_scores(tokenized_query)
    indices = np.argsort(bm25_scores)[::-1]

    return bm25_scores, indices

In [20]:
bm25_query_pipeline(query)

(array([3.68324933, 0.        , 1.16756488, 6.2959442 , 1.58776869,
        2.64665068, 0.        , 3.95040043, 0.        , 1.36335155,
        3.55307587, 1.06050578, 2.72203545, 0.68978639, 5.35570307,
        1.08198647, 1.0722069 , 0.88145497, 0.69584199, 0.80733947,
        0.        , 2.06048636, 0.        ]),
 array([ 3, 14,  7,  0, 10, 12,  5, 21,  4,  9,  2, 15, 16, 11, 17, 19, 18,
        13,  8,  6, 20,  1, 22], dtype=int64))

### 8. Applying fusion scoring
**`α * xi + (1 - α) * yi`**<br><br>
...where `xi` is score of the ith chunk from the bert model and `yi` is score of the ith chunk from bm25.

In [ ]:
# function to normalize the scores
def normalize_scores(scores):
    min_s = np.min(scores)
    max_s = np.max(scores)
    return (scores - min_s) / (max_s - min_s) if max_s > min_s else scores

# function to fuse the scores
def fused_scores(query, alpha = 0.3, top_l = 15):
    bm25_scores, bm25_indices = bm25_query_pipeline(query)
    bert_scores, bert_indices = bert_final_scores(query)
    
    # create arrays to hold scores aligned by document index
    num_docs = len(bm25_scores)  # should be same as bert_scores length
    bm25_aligned = np.zeros(num_docs)
    bert_aligned = np.zeros(num_docs)
    
    # align bm25 scores (indices are original document indices)
    for idx, score in zip(bm25_indices, bm25_scores):
        bm25_aligned[idx] = score

    # align BERT scores
    for idx, score in zip(bert_indices, bert_scores):
        bert_aligned[idx] = score

    # normalize
    bm25_norm = normalize_scores(bm25_aligned)
    bert_norm = normalize_scores(bert_aligned)

    # fuse
    fused = alpha * bm25_norm + (1 - alpha) * bert_norm

    # top-L indices by fused score
    top_indices = np.argsort(fused)[::-1][:top_l]

    return top_indices

best = fused_scores(query)

### 9. Get final context

In [22]:
final_context = ''
for idx in best:
    # remove unnecessary dots
    final_string = re.sub(r'\.{2,}', '.', chunks[idx])
    final_context += f"{final_string.strip()} (idx = {idx})\n\n"

final_context

'TEXT DATA: fervi.com TEXT DATA: The main switch with safety key is located on the left side of the electric panel, and it is useful to prevent unauthorised use of the machine. To power the machine (see  Figure 16 ), in fact, it is necessary to: \uf0b7 insert the safety key into the lock to the left of the switch (marked R.L.); \uf0b7 turn the key clockwise and simultaneously turn the main switch to "I", by turning it clockwise. TEXT DATA: Figure 16 - Main switch. TEXT DATA: Confirm the white power light has turned on, found on the electrical panel (marked POWER). TEXT DATA: 9.1.2 Electric and control panel buttons and lights TEXT DATA: Figure 17 – Electrical and control panel. TEXT DATA: The electrical panel has the following controls: \uf0b7 RED MUSHROOM :  Emergency button (red mushroom). \uf0b7 WHITE WARNING LIGHT :  Indicates the presence of voltage \uf0b7 BLACK SELECTOR :  Coolant pump selector \uf0b7 GREEN BUTTON  : Gears application button TEXT DATA: Page 28 of 84 TEXT DATA: MA

### 10. Setting up LLM A that returns indexes of top-M chunks based on the query

In [23]:
def prompt_rank_contexts(query, final_context, top_m = 5):
    context_string = ""
    for i, ctx in enumerate(final_context):
        context_string += f"Context {i+1}:\n{ctx.strip()}\n\n"

    return f"""
        You are a highly skilled AI assistant that ranks technical contexts from a machinery operations and maintenance manual.

        Given a user question and several candidate context excerpts from the manual, rank the top-L most relevant ones for answering the question. Relevance means how well the context can be used to answer the question **accurately and directly**.

        Each context ends with a tag in the format **(idx = N)**. Use this identifier to reference the context when deciding relevance.

        User Question:
        {query}

        Candidate Contexts:
        {context_string}

        Return only the `idx` values of the top {top_m} most relevant contexts, in descending order of relevance (most relevant first). Format your response like this:
        22, 8, 15, 4, 31
        """

def llm_a(query, final_context): # mistral might not be the best but no option
    prompt = prompt_rank_contexts(query, final_context)

    try:
        response = ollama.generate(
            model = 'mistral', 
            prompt = prompt,
            stream = False
        )
        output = response['response'].strip()

        # parse the returned string for integer idx values
        ranked_indexes = [int(idx.strip()) for idx in output.split(",") if idx.strip().isdigit()]
        return ranked_indexes[:5]

    except Exception as e:
        print("Error using Ollama (Python client):", str(e))
        return []

In [24]:
# get ids of top-M chunks
top_ids = llm_a(query, final_context)
print(f"IDs of top-M chunks are: {top_ids}")

IDs of top-M chunks are: [21, 17, 20, 6, 5]


### 11. Get final context

In [25]:
final_context = ''
for idx in top_ids:
    final_context += chunks[idx]

In [26]:
final_context

'TEXT DATA: Button Machine TEXT DATA: 0 Multifunction milling machine TEXT DATA: 1 Universal milling machine TEXT DATA: 2 Process unloader TEXT DATA: 3 Lathe TEXT DATA: \uf0f2 ENT TEXT DATA: - Press the buttons and to move to the next step. TEXT DATA: - As you are using the  l a t h e,  s e lect number 3, then the instrument will display TEXT DATA: only the X and Y dimensions necessary for turning operations TEXT DATA: Page 37 of 84TEXT DATA: fervi.com TEXT DATA: F TABLE DATA: [[\'\']] TEXT DATA: I TEXT DATA: H TEXT DATA: G TEXT DATA: Figure 19 - Feed adjustment knobs. TEXT DATA: The panel for automatic feed speed adjustment (see  Figure 19 ), is positioned at the left part of the machine, under the spindle rotation speed adjustment panel. The different speeds are selected via 4 knob switches which control the gear speed change (ref.  F ,  G ,  H ,  I  in  Figure 19 ). There is also a fifth knob for selecting the feed direction of the carriage (ref.  E  in  Figure 19 ). TEXT DATA: Thre

### 12. Setting up LLM B for response generation

In [27]:
def llm_b(prompt):
    client = Groq(
        api_key = os.getenv("GROQ_API_KEY"),
    )

    chat_completion = client.chat.completions.create(
        model = "llama-3.3-70b-versatile",
        # model = "llama3-70b-8192",
        # model = "mistral-saba-24b",
        messages = [
            {
                "role": "system",
                "content": "You are an expert technical assistant specialized in interpreting operations and maintenance manuals for machinery."
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        temperature = 0.5,
        max_tokens = 5640,
        top_p = 1,
        stream = True,
    )

    for chunk in chat_completion:
        content = chunk.choices[0].delta.content
        if content:
            print(content, end = '', flush = True)  # print to console without newline, flush immediately
            time.sleep(0.01)  # optional tiny delay for typewriter effect
    

def prompt(query, context):
    return f"""
        You are an expert technical assistant specialized in interpreting operations and maintenance manuals for machinery.

        Given the user question and the relevant extracted context from the manual:

        - Provide a clear, precise, and factual answer to the question.
        - Base your response strictly on the provided context; do not guess beyond it.
        - If the context does not contain enough information, indicate that the answer is not available in the manual or that the context is not sufficient.
        - Keep the answer professional, concise, and focused on practical instructions.
        - Each section of the context begins with a tag: either 'TEXT DATA' or 'TABLE DATA'.
        - 'TEXT DATA' represents plain, unstructured text. 'TABLE DATA' represents information extracted from a table and flattened into a list format.
        - The 'TABLE DATA' is structured as a list of rows, where each row is a list containing the column values in order. The format is as follows: [[column 1 value, column 2 value, ...], [column 1 value, column 2 value, ...], ...]
        
        User Question:
        {query}

        Context from Manual:
        {context}
        """

### 13. Inference

In [28]:
prompt = prompt(query, final_context) # go to section number 5 to change query
llm_b(prompt)

The manual does not provide information on unplugging the power cord of the machine from the power outlet. The context provided does not contain any instructions or guidelines related to this specific topic. Therefore, the answer is not available in the manual.